# NAIVE LEARNING

In [ ]:
import wandb

from sage.config import *
from sage.training import runner
from utils.misc import seed_everything

In [ ]:
cfg = load_config()
seed_everything(cfg.seed)
cfg.registration = 'mni'
cfg.unused_src = ['IXI', 'Dallas', 'Oasis1']
cfg.plot = False

cfg.augment = False
cfg.epochs = 500
cfg.model_name = 'resnet_no_maxpool'
cfg.early_patience = 30
# cfg.start_channels = 8

In [ ]:
name = 'MNI / OAS3 ONLY / EXTRA_EPOCHS / Resnet_no_maxpool'

In [ ]:
wandb.login()
wandb.init(project='3d_smri',
           config=vars(cfg),
           name=name
  )

In [ ]:
model = runner.run(cfg)

In [ ]:
wandb.finish()

# UNLEARNING

In [1]:
import wandb

from sage.config import *
from sage.training import unlearner
from utils.misc import seed_everything

In [2]:
cfg = load_config()
seed_everything(cfg.seed)
cfg.registration = 'mni'
cfg.unused_src = []

cfg.unlearn = True
cfg.unlearn_cfg.encoder.name = 'resnet'
cfg.unlearn_cfg.domainer.num_dbs = 4 - len(cfg.unused_src)

In [3]:
name = 'RESNET(16) 2DB conf=NoDetach'

In [4]:
wandb.login()
wandb.init(project='3d_smri_unlearning',
           config=vars(cfg),
           name=name
  )

wandb: Currently logged in as: 1pha (use `wandb login --relogin` to force relogin)


In [5]:
unlearner.run(cfg)

Use cuda:0 as a device.
{'model_depth': 10, 'n_classes': 1, 'n_input_channels': 1, 'shortcut_type': 'A', 'conv1_t_size': 7, 'conv1_t_stride': 2, 'no_max_pool': False, 'start_channels': 16}
Output from encoder is 128.
TOTAL TRAIN 1141 | VALID 286
Epoch 1 / 100, BEST MAE inf
[train] 66.2 sec [valid] 9.8 sec 
	TRAIN :: 
	REG  3442.01 | DOM     1.38 | CONF  219.96
	ACC      0.34 | AUC      0.55
	RMSE    58.64 | MAE     55.60 | R2      -8.88 | CORR     0.11
	VALID :: 
	REG  3307.74 | DOM     1.38 | CONF  220.56
	ACC      0.56 | AUC      0.70
	RMSE    57.53 | MAE     54.21 | R2      -7.92 | CORR     0.07
Saving ...
Epoch 2 / 100, BEST MAE 54.213
[train] 63.3 sec [valid] 8.9 sec 
	TRAIN :: 
	REG  3292.71 | DOM     1.37 | CONF  220.85
	ACC      0.41 | AUC      0.66
	RMSE    57.36 | MAE     54.27 | R2      -8.45 | CORR     0.15
	VALID :: 
	REG  3032.94 | DOM     1.36 | CONF  222.77
	ACC      0.39 | AUC      0.81
	RMSE    55.09 | MAE     51.75 | R2      -7.18 | CORR     0.64
Saving ...
Epoch 3 /

[train] 74.7 sec [valid] 12.0 sec 
	TRAIN :: 
	REG    98.10 | DOM     1.31 | CONF  297.75
	ACC      0.39 | AUC      0.71
	RMSE     9.92 | MAE      8.12 | R2       0.72 | CORR     0.92
	VALID :: 
	REG   144.74 | DOM     1.30 | CONF  283.78
	ACC      0.39 | AUC      0.73
	RMSE    12.03 | MAE     10.39 | R2       0.61 | CORR     0.83
Epoch 24 / 100, BEST MAE 8.707
[train] 65.8 sec [valid] 10.0 sec 
	TRAIN :: 
	REG    95.48 | DOM     1.31 | CONF  298.04
	ACC      0.39 | AUC      0.67
	RMSE     9.79 | MAE      7.99 | R2       0.72 | CORR     0.92
	VALID :: 
	REG   156.94 | DOM     1.30 | CONF  285.50
	ACC      0.39 | AUC      0.68
	RMSE    12.53 | MAE     10.82 | R2       0.58 | CORR     0.80
Epoch 25 / 100, BEST MAE 8.707
[train] 66.4 sec [valid] 8.9 sec 
	TRAIN :: 
	REG   101.70 | DOM     1.32 | CONF  301.00
	ACC      0.39 | AUC      0.64
	RMSE    10.10 | MAE      8.21 | R2       0.71 | CORR     0.91
	VALID :: 
	REG   198.30 | DOM     1.30 | CONF  279.64
	ACC      0.39 | AUC      0.67
	RM

[train] 52.8 sec [valid] 7.2 sec 
	TRAIN :: 
	REG    96.30 | DOM     1.31 | CONF  298.76
	ACC      0.39 | AUC      0.68
	RMSE     9.85 | MAE      7.93 | R2       0.72 | CORR     0.93
	VALID :: 
	REG   138.51 | DOM     1.30 | CONF  281.65
	ACC      0.39 | AUC      0.72
	RMSE    11.77 | MAE     10.20 | R2       0.63 | CORR     0.86
Epoch 47 / 100, BEST MAE 8.707
[train] 59.2 sec [valid] 7.8 sec 
	TRAIN :: 
	REG    95.33 | DOM     1.31 | CONF  299.50
	ACC      0.39 | AUC      0.66
	RMSE     9.80 | MAE      7.91 | R2       0.72 | CORR     0.93
	VALID :: 
	REG   125.85 | DOM     1.29 | CONF  296.78
	ACC      0.39 | AUC      0.72
	RMSE    11.22 | MAE      9.22 | R2       0.66 | CORR     0.83
Epoch 48 / 100, BEST MAE 8.707
[train] 54.3 sec [valid] 6.3 sec 
	TRAIN :: 
	REG    97.06 | DOM     1.31 | CONF  300.81
	ACC      0.39 | AUC      0.67
	RMSE     9.89 | MAE      7.95 | R2       0.72 | CORR     0.93
	VALID :: 
	REG   120.24 | DOM     1.30 | CONF  291.94
	ACC      0.39 | AUC      0.72
	RMSE

[train] 50.1 sec [valid] 5.9 sec 
	TRAIN :: 
	REG   100.31 | DOM     1.30 | CONF  296.21
	ACC      0.39 | AUC      0.67
	RMSE    10.04 | MAE      8.14 | R2       0.71 | CORR     0.93
	VALID :: 
	REG   132.64 | DOM     1.29 | CONF  290.40
	ACC      0.39 | AUC      0.69
	RMSE    11.53 | MAE      9.61 | R2       0.64 | CORR     0.88
Epoch 70 / 100, BEST MAE 8.707
[train] 48.7 sec [valid] 5.7 sec 
	TRAIN :: 
	REG   102.04 | DOM     1.30 | CONF  296.33
	ACC      0.39 | AUC      0.67
	RMSE    10.13 | MAE      8.23 | R2       0.71 | CORR     0.93
	VALID :: 
	REG   115.59 | DOM     1.29 | CONF  292.71
	ACC      0.39 | AUC      0.70
	RMSE    10.76 | MAE      8.84 | R2       0.69 | CORR     0.89
Epoch 71 / 100, BEST MAE 8.707
[train] 49.6 sec [valid] 5.8 sec 
	TRAIN :: 
	REG   105.18 | DOM     1.30 | CONF  298.77
	ACC      0.39 | AUC      0.67
	RMSE    10.29 | MAE      8.34 | R2       0.70 | CORR     0.92
	VALID :: 
	REG   131.15 | DOM     1.29 | CONF  287.38
	ACC      0.39 | AUC      0.69
	RMSE

[train] 54.7 sec [valid] 6.9 sec 
	TRAIN :: 
	REG   160.16 | DOM     1.29 | CONF  346.01
	ACC      0.44 | AUC      0.61
	RMSE    12.68 | MAE     10.31 | R2       0.54 | CORR     0.79
	VALID :: 
	REG   212.43 | DOM     1.25 | CONF  419.63
	ACC      0.50 | AUC      0.65
	RMSE    14.58 | MAE     11.33 | R2       0.43 | CORR     0.76
Epoch 93 / 100, BEST MAE 8.707
[train] 50.3 sec [valid] 6.7 sec 
	TRAIN :: 
	REG   143.98 | DOM     1.29 | CONF  337.59
	ACC      0.45 | AUC      0.63
	RMSE    12.03 | MAE     10.04 | R2       0.58 | CORR     0.82
	VALID :: 
	REG   141.38 | DOM     1.27 | CONF  361.40
	ACC      0.46 | AUC      0.64
	RMSE    11.90 | MAE      9.95 | R2       0.62 | CORR     0.85
Epoch 94 / 100, BEST MAE 8.707
[train] 56.9 sec [valid] 6.6 sec 
	TRAIN :: 
	REG   154.84 | DOM     1.29 | CONF  355.00
	ACC      0.43 | AUC      0.63
	RMSE    12.45 | MAE      9.98 | R2       0.55 | CORR     0.79
	VALID :: 
	REG   237.71 | DOM     1.27 | CONF  476.90
	ACC      0.41 | AUC      0.62
	RMSE

train_rmse,13.32921
train_mae,10.65152
train_r2,0.48948
train_corr,0.74381
valid_rmse,14.97144
valid_mae,11.65733
valid_r2,0.39624
valid_corr,0.75419
train_acc,0.42682
train_auc,0.62453
valid_acc,0.39161


train_rmse,█▇▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂
train_mae,█▇▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁
train_r2,▁▂▇█████████████████████████████████████
train_corr,▁▃▆▇███████████████████████████▇▇▇▇▇▇▇▄▆
valid_rmse,█▇▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂
valid_mae,█▇▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁
valid_r2,▁▃▇█████████████████████████████████████
valid_corr,▁▆██▇▇███▇▇█▇▇██████▇▇▇████████▇▇█▇▇▇▇▆▇
train_acc,▁▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▇█▇▇▇
train_auc,▁▇▆▆▇▆▆█▇█▅▆▆▇▇▆▇▇▆▇▇▆▆▇▆▇▆▆▇▆▆▅▅▄▄▂▃▄▂▄
valid_acc,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▆▂▂▂▆▂█▁


(ResNet(
   (conv1): Conv3d(1, 16, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
   (bn1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
       (bn1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
       (bn2): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
   )
   (layer2): Sequential(
     (0): BasicBlock(
       (conv1): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
       (bn1): BatchNorm3d(32, eps=1e-05, moment

--- 절취선 ---

In [6]:
import wandb

from sage.config import *
from sage.training import unlearner
from utils.misc import seed_everything

In [7]:
cfg = load_config()
seed_everything(cfg.seed)
cfg.registration = 'mni'
cfg.unused_src = []

cfg.unlearn = True
# cfg.unlearn_cfg.encoder.name = 'resnet'
cfg.unlearn_cfg.domainer.num_dbs = 4 - len(cfg.unused_src)

In [8]:
name = 'BASE 4DB conf=NoDetach'

In [9]:
wandb.login()
wandb.init(project='3d_smri_unlearning',
           config=vars(cfg),
           name=name
  )

In [10]:
unlearner.run(cfg)

Use cuda:0 as a device.
Output from encoder is 128.
TOTAL TRAIN 1141 | VALID 286
Epoch 1 / 100, BEST MAE inf
[train] 129.3 sec [valid] 7.3 sec 
	TRAIN :: 
	REG  3473.03 | DOM     1.38 | CONF  221.39
	ACC      0.37 | AUC      0.69
	RMSE    58.90 | MAE     55.88 | R2      -8.97 | CORR     0.19
	VALID :: 
	REG  3341.30 | DOM     1.37 | CONF  222.60
	ACC      0.37 | AUC      0.72
	RMSE    57.82 | MAE     54.55 | R2      -8.01 | CORR     0.26
Saving ...
Epoch 2 / 100, BEST MAE 54.554
[train] 125.8 sec [valid] 6.5 sec 
	TRAIN :: 
	REG  3371.91 | DOM     1.36 | CONF  223.16
	ACC      0.39 | AUC      0.72
	RMSE    58.04 | MAE     55.02 | R2      -8.68 | CORR     0.29
	VALID :: 
	REG  3200.58 | DOM     1.35 | CONF  225.14
	ACC      0.39 | AUC      0.77
	RMSE    56.59 | MAE     53.36 | R2      -7.63 | CORR     0.51
Saving ...
Epoch 3 / 100, BEST MAE 53.359
[train] 127.5 sec [valid] 6.5 sec 
	TRAIN :: 
	REG  3128.85 | DOM     1.34 | CONF  228.11
	ACC      0.39 | AUC      0.77
	RMSE    55.92 | MAE

[train] 129.6 sec [valid] 6.9 sec 
	TRAIN :: 
	REG   231.33 | DOM     1.30 | CONF  288.62
	ACC      0.39 | AUC      0.72
	RMSE    15.26 | MAE     12.83 | R2       0.33 | CORR     0.73
	VALID :: 
	REG   186.98 | DOM     1.29 | CONF  286.15
	ACC      0.39 | AUC      0.73
	RMSE    13.69 | MAE     11.56 | R2       0.50 | CORR     0.81
Epoch 24 / 100, BEST MAE 9.468
[train] 130.4 sec [valid] 6.7 sec 
	TRAIN :: 
	REG   235.67 | DOM     1.30 | CONF  287.38
	ACC      0.39 | AUC      0.71
	RMSE    15.40 | MAE     12.93 | R2       0.32 | CORR     0.73
	VALID :: 
	REG   154.92 | DOM     1.29 | CONF  294.65
	ACC      0.39 | AUC      0.74
	RMSE    12.46 | MAE     10.50 | R2       0.58 | CORR     0.79
Epoch 25 / 100, BEST MAE 9.468
[train] 128.4 sec [valid] 6.7 sec 
	TRAIN :: 
	REG   231.22 | DOM     1.30 | CONF  288.51
	ACC      0.39 | AUC      0.71
	RMSE    15.24 | MAE     12.79 | R2       0.33 | CORR     0.72
	VALID :: 
	REG   175.05 | DOM     1.27 | CONF  289.93
	ACC      0.39 | AUC      0.80
	R

[train] 140.6 sec [valid] 12.1 sec 
	TRAIN :: 
	REG   406.74 | DOM     1.32 | CONF  276.67
	ACC      0.39 | AUC      0.62
	RMSE    20.21 | MAE     17.61 | R2      -0.17 | CORR     0.56
	VALID :: 
	REG   236.44 | DOM     1.30 | CONF  294.82
	ACC      0.39 | AUC      0.70
	RMSE    15.38 | MAE     13.55 | R2       0.36 | CORR     0.75
Epoch 47 / 100, BEST MAE 9.468
[train] 144.1 sec [valid] 10.1 sec 
	TRAIN :: 
	REG   405.42 | DOM     1.33 | CONF  303.42
	ACC      0.39 | AUC      0.58
	RMSE    20.17 | MAE     16.89 | R2      -0.17 | CORR     0.30
	VALID :: 
	REG   444.99 | DOM     1.32 | CONF  270.32
	ACC      0.39 | AUC      0.69
	RMSE    21.11 | MAE     18.66 | R2      -0.20 | CORR     0.50
Epoch 48 / 100, BEST MAE 9.468
[train] 145.2 sec [valid] 9.6 sec 
	TRAIN :: 
	REG   434.25 | DOM     1.33 | CONF  278.10
	ACC      0.39 | AUC      0.59
	RMSE    20.89 | MAE     18.12 | R2      -0.25 | CORR     0.38
	VALID :: 
	REG   417.15 | DOM     1.31 | CONF  270.84
	ACC      0.39 | AUC      0.67


[train] 143.5 sec [valid] 7.3 sec 
	TRAIN :: 
	REG   342.81 | DOM     1.34 | CONF  291.61
	ACC      0.39 | AUC      0.51
	RMSE    18.54 | MAE     15.65 | R2       0.01 | CORR     0.28
	VALID :: 
	REG   365.35 | DOM     1.33 | CONF  292.49
	ACC      0.39 | AUC      0.61
	RMSE    19.12 | MAE     16.19 | R2       0.02 | CORR     0.24
Epoch 70 / 100, BEST MAE 9.468
[train] 144.6 sec [valid] 10.9 sec 
	TRAIN :: 
	REG   343.28 | DOM     1.34 | CONF  292.62
	ACC      0.39 | AUC      0.52
	RMSE    18.54 | MAE     15.59 | R2       0.01 | CORR     0.27
	VALID :: 
	REG   366.89 | DOM     1.33 | CONF  293.98
	ACC      0.39 | AUC      0.61
	RMSE    19.16 | MAE     16.17 | R2       0.01 | CORR     0.25
Epoch 71 / 100, BEST MAE 9.468
[train] 140.9 sec [valid] 7.5 sec 
	TRAIN :: 
	REG   343.99 | DOM     1.34 | CONF  294.86
	ACC      0.39 | AUC      0.49
	RMSE    18.56 | MAE     15.55 | R2       0.01 | CORR     0.23
	VALID :: 
	REG   368.29 | DOM     1.33 | CONF  295.57
	ACC      0.39 | AUC      0.61
	

[train] 128.5 sec [valid] 6.7 sec 
	TRAIN :: 
	REG   347.02 | DOM     1.34 | CONF  310.63
	ACC      0.39 | AUC      0.50
	RMSE    18.64 | MAE     15.43 | R2       0.00 | CORR     0.06
	VALID :: 
	REG   371.60 | DOM     1.33 | CONF  311.26
	ACC      0.39 | AUC      0.53
	RMSE    19.28 | MAE     16.04 | R2      -0.00 | CORR     0.11
Epoch 93 / 100, BEST MAE 9.468
[train] 128.5 sec [valid] 6.8 sec 
	TRAIN :: 
	REG   346.66 | DOM     1.34 | CONF  310.58
	ACC      0.39 | AUC      0.50
	RMSE    18.63 | MAE     15.43 | R2       0.00 | CORR     0.07
	VALID :: 
	REG   371.52 | DOM     1.33 | CONF  311.75
	ACC      0.39 | AUC      0.54
	RMSE    19.28 | MAE     16.04 | R2      -0.00 | CORR     0.12
Epoch 94 / 100, BEST MAE 9.468
[train] 128.6 sec [valid] 6.7 sec 
	TRAIN :: 
	REG   346.74 | DOM     1.34 | CONF  311.60
	ACC      0.39 | AUC      0.50
	RMSE    18.63 | MAE     15.43 | R2       0.00 | CORR     0.07
	VALID :: 
	REG   371.66 | DOM     1.33 | CONF  311.92
	ACC      0.39 | AUC      0.53
	R

train_rmse,18.63969
train_mae,15.43146
train_r2,0.00166
train_corr,0.05072
valid_rmse,19.28482
valid_mae,16.03893
valid_r2,-0.00177
valid_corr,0.11095
train_acc,0.38563
train_auc,0.49571
valid_acc,0.39161


train_rmse,██▄▃▂▂▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
train_mae,██▄▃▂▂▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train_r2,▁▂▆▇▇██████████▇▇▇██████████████████████
train_corr,▂▄▆▇▇████▇▇▇▆▆▅▄▅▆▆▅▅▅▅▄▅▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁
valid_rmse,█▇▅▃▃▃▂▂▁▁▁▁▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
valid_mae,█▇▄▃▃▃▂▂▁▁▁▁▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
valid_r2,▁▃▆▇▇▇█▇█████▇█▇▇██▇████████████████████
valid_corr,▂▆▇▇▇█▇▅▇████▇▇▅▆▆▇▆▅▆▆▆▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁
train_acc,▁███████████████████████████████████████
train_auc,▆███▇▇▇▆▇▇▆▆▅▅▅▄▄▄▄▄▄▄▄▄▄▃▂▂▂▂▁▂▂▂▁▂▁▂▁▁
valid_acc,▁███████████████████████████████████████


(VanillaConv(
   (feature_extractor): Sequential(
     (0): StrideBlock(
       (conv1): Conv3d(1, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
       (bn1): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (conv2): Conv3d(8, 8, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
       (bn2): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU()
     )
     (1): StrideBlock(
       (conv1): Conv3d(8, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
       (bn1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (conv2): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
       (bn2): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU()
     )
     (2): StrideBlock(
       (conv1): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))